In [137]:
import pandas as pd
import numpy as np
import os
import time
import re
import csv
import pickle
from collections import Counter
import matplotlib.pyplot as plt
from hdfs import InsecureClient
from sklearn.feature_extraction.text import CountVectorizer
%matplotlib inline
from chardet.universaldetector import UniversalDetector
client = InsecureClient('http://bat354.cbs.vn:50070')

In [138]:
current_root = '/home/polynote/notebooks/anhdt157/classify-use-ml/end-to-end-final-predict/'
root_dir = '/home/polynote/notebooks/anhdt157/'

In [139]:
MONTH = '202001' # MODIFY

In [140]:
TREE_VERSION = '8' #MODIFY

In [141]:
###### PATH_INTEREST_TREE = "/data-product/user-profile/settings/interest_keywords_v7.csv"
PATH_INTEREST_TREE = current_root + 'old_logic/tree/Interest_tree_v8_20200220.xlsx'
PATH_DINFO = root_dir + 'crawl_data/output_dinfo_each_month/dinfo_' + MONTH + '.csv'
PATH_MODEL = current_root + 'old_logic/model_bow/model_' + MONTH

# RESULT
PATH_RESULT_OLD_LOGIC = current_root + 'old_logic/result/result_' + MONTH + '.csv' 

# path này có thể thay đổi, nên không thể sử dụng MODE được
PATH_DATA_EN = current_root + 'input_data/en-data-test-' + MONTH + '.csv'
PATH_DATA_VI = current_root + 'input_data/vi-data-test-' + MONTH + '.csv'

In [142]:
# RESULT
output_test_cant_parsing = current_root + 'train_test_data/test/dinfo-data-test-cant-parsing-' + MONTH + '.csv'
output_test_spam = current_root + 'train_test_data/test/data-test-spam-' + MONTH + '.csv'
output_test_not_spam = current_root + 'train_test_data/test/data-test-not-spam-' + MONTH + '.csv'

In [143]:
# path = "/home/datamon/anhns3/data/from_hdfs/crawl_all_20190518/"
# df = pd.read_parquet(path)

In [144]:
# df

In [145]:
# #preprocess functions
# from pyvi.ViTokenizer import tokenize
# import string
# import re
# import langdetect

# from nltk.corpus import stopwords
# from nltk.tokenize import word_tokenize
# from nltk.stem import WordNetLemmatizer
# lm = WordNetLemmatizer()

# #read vietnamese stopwords
# with open("/home/datamon/anhns3/stopwords.txt") as f:
#     sw_vn = [line.strip() for line in f.readlines()]

# def process_sentence_vn(sentence):
#     """
#     process sentence, including:
#         remove any domain appeared
#         remove punctuations
#         Vietnamese word segmentation
#         remove VN stopwords
#     """
#     nd = ' '.join([word for word in sentence.split() if not re.search('[\w\d]+\.[\w\d]+', word) and len(word)<30])
#     npc = ''.join([s for s in nd if s not in string.punctuation])
#     tok = tokenize(npc)
#     res = ' '.join([word for word in tok.split() if (word not in sw_vn) & len(word) < 30])
#     return res

# def process_sentence_others(sentence):
#     nd = ' '.join([word for word in sentence.split() if not re.search('[\w\d]+\.[\w\d]+', word)])
#     tok = word_tokenize(nd)
#     nsw = ' '.join([w for w in tok if w not in stopwords.words("english")])
#     lemma = ' '.join([lm.lemmatize(w) for w in nsw.split()])
#     res = ''.join([c for c in lemma if c not in string.punctuation])
#     res = re.sub(' +', ' ', res)
#     return res

# def process_tit_desc(sent):    
#     try:
#         lang = langdetect.detect(sent)
#     except:
#         lang = "undef"
#     if lang == "vi":
#         res = process_sentence_vn(sent.lower())
#     else:
#         res = process_sentence_others(sent.lower())
#     return lang, res

In [146]:
# %%time
# df.iloc[:10000]["tit_des"].apply(process_tit_desc)

In [147]:
#  print("currently at batch {} over {}, processing time for this batch: {}s, total {}s"
#           .format(i, total_iters, round(time.time()-tic, 2), round(time.time()-toc, 2)))

In [148]:
# #preprocess:
# batch_size = 20000
# #df = df.drop_duplicates(subset="domain_clean")
# total_iters = int(len(df)/batch_size)
# tic = time.time()
# toc = time.time()
# for i in range(total_iters+1):
#     start_idx = batch_size*i
#     stop_idx = min(batch_size*(i+1), len(df))
#     tmp = df.iloc[start_idx:stop_idx]
#     tmp["lang"], tmp["desc"] = zip(*tmp["tit_des"].map(process_tit_desc))
#     fname = "preprocess_"+str(start_idx)+"_"+str(stop_idx) 
#     tmp.to_csv("/home/datamon/anhns3/result/20190518_domains_preprocess/"+fname, 
#                columns=["domain", "domain_clean", "lang", "desc"])
#     print("currently at batch {} over {}, processing time for this batch: {}s, total {}s"
#           .format(i, total_iters, round(time.time()-tic, 2), round(time.time()-toc, 2)))
#     tic = time.time()

In [149]:
# %%time
# path = "/home/datamon/anhns3/result/20190518_domains_preprocess/"
# list_dirs = os.listdir(path)
# list_dirs = [f for f in list_dirs if re.search("^preprocess", f)]
# cnt = []
# for fname in list_dirs:
#     content = pd.read_csv(os.path.join(path, fname), index_col=0)#, header=True)
#     cnt.append(content)
# data = pd.concat(cnt, axis=0).sort_index()

In [150]:
# data

In [151]:
# #detect blocked domains
# blocked = ["phim03.blogspot.com", "freedomfightersforamerica.com", "cap3phim.blogspot.com", "phimtamtinhtruoitre.blogspot.com", 
#  "tlnk.io", "mobileapptracking.com", "measure.mobileapptracking.com", "cutestat.com","websiteoutlook.com"]
# def check_blocked(desc):
#     if type(desc)==str:
#         for word in blocked:
#             if word in desc:
#                 return True
#     return False

In [152]:
# %%time
# data["blocked"] = data["domain"].apply(check_blocked)

In [153]:
# data[data["blocked"]==True]

In [154]:
# %%time
# df_res = pd.merge(data, df[["tit_des", "domain"]], left_on=["domain"], right_on=["domain"])
# df_res

In [155]:
# df_res.columns = ['domain', 'domain_clean', 'lang', 'title_desc_preprocess', 'blocked', 'title_des_raw']
# df_res["domain_ID"]=df_res.index
# df_res = df_res[['domain_ID', 'domain', 'domain_clean', 'title_des_raw', 'title_desc_preprocess', 'lang', 'blocked']]
# df_res

In [156]:
# output_date = "05182019"
# output_len = str(len(df_res))
# org_path = "/home/datamon/anhns3/result/domain_info/"
# output_path = os.path.join(org_path, output_date+"_"+output_len+".csv")
# df_res.to_csv(output_path, sep="|", index=False)

In [157]:
# df_data = pd.read_csv("/home/datamon/anhns3/result/domain_info/domain_info_05182019_5096082.csv", 
#                       delimiter="|")

In [158]:
# df_data.columns

In [159]:
# df_unique = df_data[df_data["blocked"]==False].drop_duplicates(subset="domain_clean")[["domain_ID", "domain_clean", "title_desc_preprocess"]].dropna()

In [160]:
# df_unique

# DATA

In [161]:
# df_unique
# data vi và en đã được tách và xử lý
df_input_data_vi = pd.read_csv(PATH_DATA_VI, delimiter='|')
# Những trường NaN thay bằng ''
df_input_data_vi = df_input_data_vi.fillna('')
df_input_data_vi['lang'] = df_input_data_vi.apply(lambda row: 'vi', axis=1)

df_input_data_en = pd.read_csv(PATH_DATA_EN, delimiter='|')
# Những trường NaN thay bằng ''
df_input_data_en = df_input_data_en.fillna('')
df_input_data_en['lang'] = df_input_data_en.apply(lambda row: 'en', axis=1)

#merge
df_unique = df_input_data_vi.append(df_input_data_en, ignore_index=True)

In [162]:
len(df_input_data_vi)

150266

In [163]:
len(df_input_data_en)

257454

In [164]:
df_info = pd.read_csv(PATH_DINFO, delimiter='\t')

In [165]:
len(df_unique)

407720

In [166]:
# bizz.vn
# df_unique
df_unique[df_unique['domain_clean'].astype(str).str.contains('bizz.vn')].head(5)

,domain_clean,title_desc_preprocess,lang
365,balotuixach.bizz.vn,công_ty_tnhh homy việt nam chuyên may balo túi...,vi
1113,nppngoctrai.bizz.vn,nhà phân_phối trần_đình ngọc_trai chuyên cung_...,vi
1264,hungphat.bizz.vn,công_ty cổ_phần sản_xuất và thương_mại hùng ph...,vi
2063,onggioanhminh.bizz.vn,công_ty_tnhh dịch_vụ cơ_khí xây_dựng anh_minh ...,vi
2085,ketcautheptnp.bizz.vn,công_ty_tnhh mtv cơ_khí thương_mại tuấn nam_ph...,vi


In [167]:
len(df_unique[df_unique['domain_clean'].astype(str).str.contains('bizz.vn')])

285

In [168]:
df_bizz = df_info[df_info['domain_clean'].astype(str).str.contains('bizz.vn')]
df_bizz

,domain,domain_redirect,domain_clean,redirected,title_des_raw,title_desc_preprocess,lang,date
8251,balotuixach.bizz.vn,balotuixach.bizz.vn,balotuixach.bizz.vn,1,"CÔNG TY TNHH HOMY VIỆT NAM Chuyên may balo, tú...",công_ty_tnhh homy việt nam chuyên may balo túi...,vi,202001
25897,nppngoctrai.bizz.vn,nppngoctrai.bizz.vn,nppngoctrai.bizz.vn,1,Nhà Phân Phối Trần Đình Ngọc Trai Chuyên cung ...,nhà phân_phối trần_đình ngọc_trai chuyên cung_...,vi,202001
28757,hungphat.bizz.vn,hungphat.bizz.vn,hungphat.bizz.vn,1,Công Ty Cổ Phần Sản Xuất Và Thương Mại Hùng Ph...,công_ty cổ_phần sản_xuất và thương_mại hùng ph...,vi,202001
46088,onggioanhminh.bizz.vn,onggioanhminh.bizz.vn,onggioanhminh.bizz.vn,1,Công Ty TNHH Dịch Vụ Cơ Khí Xây Dựng Anh Minh ...,công_ty_tnhh dịch_vụ cơ_khí xây_dựng anh_minh ...,vi,202001
46586,ketcautheptnp.bizz.vn,ketcautheptnp.bizz.vn,ketcautheptnp.bizz.vn,1,Công Ty TNHH MTV Cơ Khí Thương Mại Tuấn Nam Ph...,công_ty_tnhh mtv cơ_khí thương_mại tuấn nam_ph...,vi,202001
...,...,...,...,...,...,...,...,...
3408088,thanhvuong.bizz.vn,thanhvuong.bizz.vn,thanhvuong.bizz.vn,1,CÔNG TY TNHH DỆT MAY THÀNH VƯỢNG,công_ty_tnhh dệt_may thành vượng,vi,202001
3408424,www.khanhlinh.bizz.vn,www.khanhlinh.bizz.vn,www.khanhlinh.bizz.vn,1,CÔNG TY CỔ PHẦN DỊCH VỤ THƯƠNG MẠI VÀ SẢN XUẤT...,công_ty cổ_phần dịch_vụ thương_mại và sản_xuất...,vi,202001
3413745,thienthuanthao.bizz.vn,thienthuanthao.bizz.vn,thienthuanthao.bizz.vn,1,CÔNG TY TNHH TƯ VẤN THIẾT KẾ & XÂY DỰNG THIÊN ...,công_ty_tnhh tư_vấn thiết_kế xây_dựng thiên_th...,vi,202001
3426157,anhduongjsc.bizz.vn,anhduongjsc.bizz.vn,anhduongjsc.bizz.vn,1,Công Ty Cổ Phần Dịch Vụ Ánh Dương VỚI ÁNH DƯƠN...,công_ty cổ_phần dịch_vụ ánh dương với ánh dươn...,vi,202001


In [169]:
# df_input_data_vi_1 = df_unique.drop_duplicates(subset=['domain_clean', 'title_desc_preprocess', 'lang'], keep=False)

In [170]:
# len(df_input_data_vi_1)

In [171]:
df_unique.head(5)

,domain_clean,title_desc_preprocess,lang
0,ketoanthuecanthobantin.blogspot.com,bản tin thuế công_ty_tnhh tây nam dịch_vụ báo_...,vi
1,raovattiengiang247.com,rao vat tien_giang mua ban tien_giang rao vat ...,vi
2,onthi.net.vn,cung_cấp kiến_thức lịch_sử việt nam và thế_giớ...,vi
3,xiudunvietnam.net.vn,máy đếm tiền xiudun việt nam giá rẻ trung_tâm ...,vi
4,phukienototanphu.com,đồ_chơi xe_hơi tân phú_ngọc sang chuyên đồ_chơ...,vi


In [172]:
len(df_unique)

407720

In [173]:
df_unique = df_unique.drop_duplicates(subset=['domain_clean', 'title_desc_preprocess', 'lang'])

In [174]:
len(df_unique)

407720

In [175]:
%%time
vectorizer = CountVectorizer(ngram_range=(1,2), min_df=10)
vectorizer.fit(df_unique["title_desc_preprocess"])

CPU times: user 50.1 s, sys: 1.61 s, total: 51.7 s
Wall time: 54 s


CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=10,
                ngram_range=(1, 2), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [176]:
%%time
bow_origin = vectorizer.transform(df_unique["title_desc_preprocess"])
print("shape:", bow_origin.shape)

shape: (407720, 85257)
CPU times: user 30.2 s, sys: 582 ms, total: 30.8 s
Wall time: 32 s


In [177]:
# model_file_name = "/home/datamon/anhns3/models/BOW/0518_all_bigram_10/model"
# model_file_name = PATH_MODEL
pickle.dump(vectorizer, open(PATH_MODEL, "wb"))

In [178]:
import time

In [179]:
# model_file_path = "/home/datamon/anhns3/models/BOW/0514_all_bigram_10/model"
# domain_info_path = "/home/datamon/anhns3/result/domain_info/domain_info_05182019_5096082.csv"

tic = time.time()
print("start reading domains info ...")
# df_data = pd.read_csv(domain_info_path, 
#                       delimiter="|",
#                      usecols = ["domain_ID", "domain_clean", "title_desc_preprocess", "blocked"])
# df_unique = df_data[df_data["blocked"]==False].drop_duplicates(subset="domain_clean")[["domain_ID", "domain_clean", "title_desc_preprocess"]].dropna()
data = df_unique["title_desc_preprocess"]
print("reading domain info successfully, processing time {}s".format(round(time.time()-tic, 2)))
tic = time.time()

print("start reading vocabulary model ...")
tic = time.time()
vectorizer = pickle.load(open(PATH_MODEL, "rb"))
dictionary = vectorizer.vocabulary_
print("reading model successful, number of words: {}, processing time: {}s"
      .format(len(dictionary), round(time.time()-tic,2)))
tic = time.time()

print("transforming domain descriptions ...")
bow = vectorizer.transform(data)
print("transformation success, processing time: {}s"
      .format(round(time.time()-tic,2)))


start reading domains info ...
reading domain info successfully, processing time 0.0s
start reading vocabulary model ...
reading model successful, number of words: 85257, processing time: 2.31s
transforming domain descriptions ...
transformation success, processing time: 30.92s


In [180]:
# dictionary
# bow

In [181]:
def filter_except(except_keywords, dictionary, bow, domains, lang):
    word_idx = []
    for word in except_keywords:
        try:
            word_idx.append(dictionary[word])
        except:
            pass
    keyword_matched = bow[:,word_idx]
    number_of_matches = np.sum(keyword_matched.todense(), axis=1)
    mask = np.array(number_of_matches>=1).squeeze()
    res = domains[~mask], bow[~mask], lang[~mask]
    return res

def filtering(keywords, dictionary, bow, domains, lang, threshold=2, except_keywords=[]):
    word_idx = []
    if len(except_keywords)>0:
        domains, bow, lang = filter_except(except_keywords, dictionary, bow, domains, lang)
    for word in keywords:
        try:
            word_idx.append(dictionary[word])
        except:
            pass
    keyword_matched = bow[:,word_idx]
    number_of_matches = np.sum(keyword_matched.todense(), axis=1)
    mask = np.array(number_of_matches>=threshold).squeeze()
    res = domains[mask], bow[mask], lang[mask]
    return res

# start_idx = 10000
# stop_idx = 20000
# tmp_domains = df_unique[start_idx:stop_idx]["domain_clean"]
# tmp_bow = bow[start_idx:stop_idx]
# except_keywords = ["bách_khoa"]
# filter_except(except_keywords, dictionary, tmp_bow, tmp_domains)

In [182]:
filtering([], dictionary, bow, df_unique["domain_clean"], df_unique["lang"])

(Series([], Name: domain_clean, dtype: object),
 <0x85257 sparse matrix of type '<class 'numpy.float64'>'
 	with 0 stored elements in Compressed Sparse Row format>,
 Series([], Name: lang, dtype: object))

# PREDICT

In [183]:
# def predict_encoding(file_path_name):
#     detector = UniversalDetector()
#     with open(file_path_name, 'rb') as f:
#         for line in f:
#             detector.feed(line)
#             if detector.done:
#                 break
#         detector.close()
#     r = detector.result
#     return "Decoding encoding %s with confidence %s" % (r['encoding'], r['confidence'])

In [184]:
# x = predict_encoding(PATH_INTEREST_TREE)
# print(x)

In [185]:
# df1 = pd.read_excel(PATH_INTEREST_TREE_1, skiprows=5)
# df1.head(1)

In [186]:
# len(df1)

In [187]:
# df = pd.read_csv("tree/Interests_tree_v7_05272019.csv", skiprows=5, delimiter="|")
# df = pd.read_csv(PATH_INTEREST_TREE, skiprows=5, delimiter="|")
df = pd.read_excel(PATH_INTEREST_TREE, skiprows=5)
df.head(1)

,IID,Level 1,Unnamed: 2,Level 2,Unnamed: 4,Level 3,Unnamed: 6,Status,Level 4,Unnamed: 9,Status.1,Inference heuristics,Has_child,Logic,threshold,Revised_keywords,Except,Notes
0,A000000,Business & Industry,Kinh doanh & Các ngành nghề,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,any,1.0,"in_ấn, băng_rôn, neonsign, standee, phướn, tờ_...","phim, sex",NaN


In [188]:
len(df)

330

In [189]:
# df[df["IID"]=="A060199"]

In [190]:
import re
def deslash(word):
    return re.sub("/", "-", word)
#dictionary = {}
#bow = np.zeros((1,1))
domain_params = [dictionary, bow]
class interest():
    def __init__(self, IID, keywords, inference_heuristics, level_1, level_1_vn, level_2=None, level_2_vn=None,
                 level_3=None, level_3_vn=None, level_4=None, level_4_vn=None, checked=False, has_child=0, 
                 logic = "any", except_keywords=[], passed_threshold=0, params=domain_params, lang=None):
        self.IID = IID
        self.keywords = keywords
        self.inference_heuristics = inference_heuristics
        self.level_1 = level_1
        self.level_2 = level_2
        self.level_3 = level_3
        self.level_4 = level_4
        self.level_1_vn = level_1_vn
        self.level_2_vn = level_2_vn
        self.level_3_vn = level_3_vn
        self.level_4_vn = level_4_vn
        self.checked = checked
        self.has_child = has_child
        self.domains = None
#         self.lang = None
        self.params = params
        self.set_max_level()
        self.create_path()
        self.logic = logic.strip()
        self.except_keywords = except_keywords
        self.set_threshold(passed_threshold)
    
    def set_threshold(self, passed_threshold):
        if passed_threshold==0:
            if self.level==1:
                self.threshold = 3
            else:
                self.threshold = 2
        else:
            self.threshold = passed_threshold
            #print(self.threshold)
    
    def set_max_level(self):
        if self.level_4==None:
            if self.level_3==None:
                if self.level_2==None:
                    self.level=1
                else:
                    self.level=2
            else:
                self.level=3
        else:
            self.level=4
    
    def __str__(self):
        d = {"IID":self.IID,
            "keywords":self.keywords,
             "level":self.level,
            "level_1":self.level_1,
            "level_2":self.level_2,
            "level_3":self.level_3,
            "level_4":self.level_4,
            "level_1_vn":self.level_1_vn,
            "level_2_vn":self.level_2_vn,
            "level_3_vn":self.level_3_vn,
            "level_4_vn":self.level_4_vn,
            #"Inference_heuristics":self.inference_heuristics,
             "has_child":self.has_child,
            # "checked":self.checked,
             "logic": self.logic,
             "except": self.except_keywords,
             "threshold": self.threshold
            }
        return str(d)
    
    def __repr__(self):
        return self.__str__()
    
    def create_path(self):
        list_level = [deslash(l) for l in [self.level_1, self.level_2, self.level_3, self.level_4] if type(l)==str]
        list_dirs = list_level[:(self.level-1)]
        self.path = "/".join([d for d in list_dirs if type(d)==str])
        self.file_name = list_level[self.level-1]
        
    
    def write_result(self, root_dir, sampling=False):
        dir_path = os.path.join(root_dir, self.path)
        os.makedirs(dir_path, exist_ok=True)
        full_path = os.path.join(dir_path, self.file_name+"_domains")
        kw_path = os.path.join(dir_path, self.file_name+"_keywords")
        with open(full_path, "a") as f:
            f.write("\n".join([d for d in self.domains.values]))    
        with open(kw_path, "a") as f:
            f.write("ID:"+str(self.IID)+"\n")
            f.write("level"+str(self.level)+"\n")
            f.write("length:"+str(len(self.domains))+"\n")
            f.write("keywords:"+", ".join(self.keywords))
        if sampling:
            if len(self.domains)>=100:
                sample = [self.domains.values[i] for i in np.random.randint(0, len(self.domains.values), 100)]
                with open(full_path+"_sample", "a") as f:
                    f.write("\n".join([d for d in sample])) 
        print("result saved to {}".format(full_path))
        
    def find_parent(self):
        if self.level ==4:
            return self.IID[:-2]+"00"
        if self.level == 3:
            return self.IID[:-4]+"00"+self.IID[-2:]
        if self.level ==2:
            return self.IID[:-6]+"00"+self.IID[-4:]
        return "NONE"
       
    def set_parent_domains_bow_lang(self, parent_domains, bow, lang):
        self.parent_domains = parent_domains
        self.parent_lang = lang
        self.parent_bow = bow
        print("set parent domain for {}, length {}".format(self.IID, len(self.parent_domains)))
    
    def filter_domains(self):
        dictionary, bow = self.params
        if len(self.except_keywords)>0:
            print("{} has except keywords {}".format(self.IID, self.except_keywords))
        if self.keywords == []:
            self.domains, self.bow, self.lang = self.parent_domains, self.parent_bow, self.parent_lang
        else:
            self.domains, self.bow, self.lang = filtering(keywords=self.keywords, 
                                    dictionary=dictionary, 
                                    bow = self.parent_bow, 
                                    domains = self.parent_domains, 
                                    lang = self.parent_lang,
                                    threshold = self.threshold,
                                    except_keywords = self.except_keywords)
        print("Done filtering for {}, domains found: {}".format(self.IID, len(self.lang) ))
        #return get_domains, bow_domains

In [191]:
#parse_df
def read_tree(df):
    level_1 = None
    level_2 = None
    level_3 = None
    level_4 = None
    level_1_vn = None
    level_2_vn = None
    level_3_vn = None
    level_4_vn = None
    inference_heuristics = None
    IID = None
    has_child = 0
    checked = False
    all_interest = []
    #tic = time.time()
    for i,row in df.iterrows():
        if type(row["Level 1"])==str:
            level_1 = row["Level 1"]
            level_1_vn = row["Unnamed: 2"]
            level_2, level_3, level_4 = None, None, None
            level_2_vn, level_3_vn, level_4_vn = None, None, None
        if type(row["Level 2"])==str:
            level_2 = row["Level 2"]
            level_2_vn = row["Unnamed: 4"]
            level_3, level_4 = None, None
            level_3_vn, level_4_vn = None, None
        if type(row["Level 3"])==str:
            level_3 = row["Level 3"]
            level_3_vn = row["Unnamed: 6"]
            level_4 = None
            level_4_vn = None
        if type(row["Level 4"])==str:
            level_4 = row["Level 4"]
            level_4_vn = row["Unnamed: 9"]
        if str(row["threshold"])=='nan':
            threshold = 0
        else:
            threshold = int(row["threshold"])
        if has_child in [0,1]:
            if type(row["Revised_keywords"])==str:
                keywords = [word.strip() for word in row["Revised_keywords"].split(",")]
            elif type(row["Revised_keywords"])==float:
                keywords = []                
            if type(row["Except"])==str:
                except_keywords = [word.strip() for word in row["Except"].split(",")]
            elif type(row["Except"])==float:
                except_keywords = []
            itr = interest(inference_heuristics=row["Inference heuristics"],
                           checked=row["Status"],
                           has_child=row["Has_child"],                     
                           passed_threshold=threshold,
                           keywords=keywords,
                           IID=row["IID"],
                           level_1=level_1,
                           level_2=level_2,
                           level_3=level_3,
                           level_4=level_4,
                           level_1_vn=level_1_vn,
                           level_2_vn=level_2_vn,
                           level_3_vn=level_3_vn,
                           level_4_vn=level_4_vn, 
                           logic=str(row["Logic"]),   
                           except_keywords=except_keywords,
#                            lang=str(row["lang"])
                          )
            all_interest.append(itr)
            print("{}, nb of keywords: {}, threshold: {}". format(itr.IID, len(itr.keywords), itr.threshold))
        #tic = time.time()
    print("DONE, total number of interests read: {}". format(len(all_interest)))
    return all_interest

In [192]:
tree = read_tree(df)

A000000, nb of keywords: 208, threshold: 1
A010000, nb of keywords: 27, threshold: 1
A010100, nb of keywords: 8, threshold: 1
A010199, nb of keywords: 0, threshold: 2
A010200, nb of keywords: 19, threshold: 1
A010299, nb of keywords: 0, threshold: 2
A019900, nb of keywords: 0, threshold: 2
A019999, nb of keywords: 0, threshold: 2
A020000, nb of keywords: 35, threshold: 1
A020100, nb of keywords: 28, threshold: 1
A020199, nb of keywords: 0, threshold: 2
A020200, nb of keywords: 5, threshold: 1
A020299, nb of keywords: 0, threshold: 2
A020300, nb of keywords: 3, threshold: 1
A020399, nb of keywords: 0, threshold: 2
A029900, nb of keywords: 0, threshold: 2
A029999, nb of keywords: 0, threshold: 2
A030000, nb of keywords: 1, threshold: 2
A039900, nb of keywords: 0, threshold: 2
A039999, nb of keywords: 0, threshold: 2
A040000, nb of keywords: 2, threshold: 1
A049900, nb of keywords: 0, threshold: 2
A049999, nb of keywords: 0, threshold: 2
A050000, nb of keywords: 2, threshold: 1
A050100, n

In [193]:
def find_all_siblings(itr, tree):
    IID = itr.IID
    level = itr.level
    if level == 4:
        pat = "^"+IID[:5]+"0[^0]$"
    if level == 3:
        pat = "^"+IID[:3]+"0[^0]00$"
    if level == 2:
        pat = "^"+IID[:1]+"0[^0]0000$"
    res = []
    #print(pat)
    for itr in tree:
        if re.search(pat, itr.IID):
            res.append(itr)
    return res

In [194]:
#get domains for each interest
for itr in tree:
    if itr.has_child in [0,1]:
        if itr.level==1:
            itr.set_parent_domains_bow_lang(df_unique["domain_clean"], bow, df_unique["lang"])
        else:
            pr = [item for item in tree if item.IID == itr.find_parent()][0]
            itr.set_parent_domains_bow_lang(pr.domains, pr.bow, pr.lang)
        itr.filter_domains()
        #itr.domain_indices = itr.domains.index
    else:
        print(itr.IID, "is Other!")
        try :
            parent = [item for item in tree if item.IID == itr.find_parent()][0]
            print("Parent:", parent.IID)
            sibs = find_all_siblings(itr, tree)
            print("siblings:", "|".join([child.IID for child in sibs]))
            try:
                children_domains = pd.concat([child.domains for child in sibs])
            except ValueError as e:
                children_domains = pd.Series()

            itr.domains = parent.domains[~parent.domains.isin(children_domains)]
            itr.lang = parent.lang[~parent.domains.isin(children_domains)]
            print("Number of other domains:", len(itr.domains))
        except Exception as e:
            print("===========================")
            print(itr.IID)
            break
    print(itr.IID, itr.file_name, len(itr.domains))
    print()

set parent domain for A000000, length 407720
A000000 has except keywords ['phim', 'sex']
Done filtering for A000000, domains found: 64734
A000000 Business & Industry 64734

set parent domain for A010000, length 64734
Done filtering for A010000, domains found: 3807
A010000 Advertising 3807

set parent domain for A010100, length 3807
Done filtering for A010100, domains found: 905
A010100 Local printing services 905

A010199 is Other!
Parent: A010100
siblings: 
Number of other domains: 905
A010199 Others 905

set parent domain for A010200, length 3807
Done filtering for A010200, domains found: 2917
A010200 Online marketing 2917

A010299 is Other!
Parent: A010200
siblings: 
Number of other domains: 2917
A010299 Others 2917

A019900 is Other!
Parent: A010000
siblings: A010100|A010200
Number of other domains: 0
A019900 Others 0

A019999 is Other!
Parent: A019900
siblings: 
Number of other domains: 0
A019999 Others 0

set parent domain for A020000, length 64734
Done filtering for A020000, dom

/home/polynote/.local/lib/python3.6/site-packages/ipykernel_launcher.py:23: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/home/polynote/.local/lib/python3.6/site-packages/ipykernel_launcher.py:23: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/home/polynote/.local/lib/python3.6/site-packages/ipykernel_launcher.py:23: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/home/polynote/.local/lib/python3.6/site-packages/ipykernel_launcher.py:23: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/home/polynote/.local/lib/python

Done filtering for A040000, domains found: 4224
A040000 Construction 4224

A049900 is Other!
Parent: A040000
siblings: 
Number of other domains: 4224
A049900 Others 4224

A049999 is Other!
Parent: A049900
siblings: 
Number of other domains: 4224
A049999 Others 4224

set parent domain for A050000, length 64734
Done filtering for A050000, domains found: 12706
A050000 Design 12706

set parent domain for A050100, length 12706
A050100 has except keywords ['biển_hiệu', 'quảng_cáo', 'in_ấn', 'web', 'website', 'tiếp_thị', 'truyền_thông', 'seo', 'branding', 'content', 'truyền_thông', 'băng_rôn', 'tối_ưu', 'in_ấn', 'băng_rôn', 'neonsign', 'standee', 'phướn', 'tờ_rơi', 'biển_hiệu', 'bảng_hiệu', 'copywriter', 'contentmarketing', 'marketingbureau', 'maketing', 'webmarketing', 'leadgeneratie', 'emarketing', 'copywriting', 'ppc', 'marketer', 'wordofmouth', 'adwords', 'makerting', 'ads', 'tiếp_thị', 'marketting', 'advertising', 'seo', 'branding']
Done filtering for A050100, domains found: 2851
A050100

/home/polynote/.local/lib/python3.6/site-packages/ipykernel_launcher.py:23: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/home/polynote/.local/lib/python3.6/site-packages/ipykernel_launcher.py:23: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


Done filtering for A060104, domains found: 31
A060104 Oncology 31

set parent domain for A060105, length 3669
Done filtering for A060105, domains found: 144
A060105 Veterinary 144

set parent domain for A060106, length 3669
Done filtering for A060106, domains found: 444
A060106 General 444

set parent domain for A060107, length 3669
Done filtering for A060107, domains found: 43
A060107 Andrology 43

A060199 is Other!
Parent: A060100
siblings: A060101|A060102|A060103|A060104|A060105|A060106|A060107
Number of other domains: 1192
A060199 Others 1192

set parent domain for A060200, length 4018
Done filtering for A060200, domains found: 387
A060200 Pharmacy 387

set parent domain for A060201, length 387
Done filtering for A060201, domains found: 117
A060201 Western medicine - Pharmacy  117

set parent domain for A060202, length 387
Done filtering for A060202, domains found: 194
A060202 Traditional medicine - Pharmacy  194

A060299 is Other!
Parent: A060200
siblings: A060201|A060202
Number o

...

Number of other domains: 51
G039900 Others 51

G039999 is Other!
Parent: G039900
siblings: 
Number of other domains: 51
G039999 Others 51

set parent domain for G040000, length 12032
Done filtering for G040000, domains found: 1163
G040000 Furniture 1163

G049900 is Other!
Parent: G040000
siblings: 
Number of other domains: 1163
G049900 Others 1163

G049999 is Other!
Parent: G049900
siblings: 
Number of other domains: 1163
G049999 Others 1163

set parent domain for G050000, length 12032
Done filtering for G050000, domains found: 149
G050000 Toys 149

set parent domain for G050100, length 149
Done filtering for G050100, domains found: 31
G050100 Children's toys 31

G050199 is Other!
Parent: G050100
siblings: 
Number of other domains: 31
G050199 Others 31

set parent domain for G050200, length 149
Done filtering for G050200, domains found: 37
G050200 Adult's toy 37

G050299 is Other!
Parent: G050200
siblings: 
Number of other domains: 37
G050299 Others 37

G990000 is Other!
Parent: G00000

/home/polynote/.local/lib/python3.6/site-packages/ipykernel_launcher.py:23: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/home/polynote/.local/lib/python3.6/site-packages/ipykernel_launcher.py:23: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/home/polynote/.local/lib/python3.6/site-packages/ipykernel_launcher.py:23: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


Done filtering for H000000, domains found: 4071
H000000 Technology 4071

set parent domain for H010000, length 4071
Done filtering for H010000, domains found: 1880
H010000 Computer 1880

set parent domain for H010100, length 1880
Done filtering for H010100, domains found: 25
H010100 Cloud 25

H010199 is Other!
Parent: H010100
siblings: 
Number of other domains: 25
H010199 Others 25

set parent domain for H010200, length 1880
Done filtering for H010200, domains found: 1048
H010200 PC & Laptop & Hardware 1048

H010299 is Other!
Parent: H010200
siblings: 
Number of other domains: 1048
H010299 Others 1048

set parent domain for H010300, length 1880
Done filtering for H010300, domains found: 295
H010300 Software 295

H010399 is Other!
Parent: H010300
siblings: 
Number of other domains: 295
H010399 Others 295

H019900 is Other!
Parent: H010000
siblings: H010100|H010200|H010300
Number of other domains: 546
H019900 Others 546

H019999 is Other!
Parent: H019900
siblings: 
Number of other domain

/home/polynote/.local/lib/python3.6/site-packages/ipykernel_launcher.py:23: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/home/polynote/.local/lib/python3.6/site-packages/ipykernel_launcher.py:23: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/home/polynote/.local/lib/python3.6/site-packages/ipykernel_launcher.py:23: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/home/polynote/.local/lib/python3.6/site-packages/ipykernel_launcher.py:23: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/home/polynote/.local/lib/python

Number of other domains: 1391
H039999 Others 1391

H990000 is Other!
Parent: H000000
siblings: H010000|H020000|H030000
Number of other domains: 295
H990000 Others 295

H999900 is Other!
Parent: H990000
siblings: 
Number of other domains: 295
H999900 Others 295

H999999 is Other!
Parent: H999900
siblings: 
Number of other domains: 295
H999999 Others 295



In [195]:
[(itr.IID, len(itr.domains), len(itr.lang)) for itr in tree if itr.level==4]

[('A010199', 905, 905),
 ('A010299', 2917, 2917),
 ('A019999', 0, 0),
 ('A020199', 1891, 1891),
 ('A020299', 106, 106),
 ('A020399', 239, 239),
 ('A029999', 23, 23),
 ('A039999', 0, 0),
 ('A049999', 4224, 4224),
 ('A050199', 2851, 2851),
 ('A050299', 1770, 1770),
 ('A059999', 8085, 8085),
 ('A060101', 1172, 1172),
 ('A060102', 664, 664),
 ('A060103', 274, 274),
 ('A060104', 31, 31),
 ('A060105', 144, 144),
 ('A060106', 444, 444),
 ('A060107', 43, 43),
 ('A060199', 1192, 1192),
 ('A060201', 117, 117),
 ('A060202', 194, 194),
 ('A060299', 77, 77),
 ('A060301', 8, 8),
 ('A060399', 217, 217),
 ('A069999', 19, 19),
 ('A070101', 95, 95),
 ('A070102', 147, 147),
 ('A070103', 74, 74),
 ('A070104', 105, 105),
 ('A070105', 560, 560),
 ('A070299', 55, 55),
 ('A070301', 147, 147),
 ('A070302', 193, 193),
 ('A070399', 205, 205),
 ('A070401', 1137, 1137),
 ('A079999', 733, 733),
 ('A089999', 4331, 4331),
 ('A090199', 1372, 1372),
 ('A090299', 0, 0),
 ('A090399', 431, 431),
 ('A099999', 2357, 2357),


In [196]:
# fname = "interest_domain_v7_05272019.csv"
# output_file = os.path.join('output_domains', fname)
from datetime import datetime
def get_time_now():
    return str(int(datetime.timestamp(datetime.now())))

def get_parent_from_lv4(ID):
    level_3 = ID[:-2]+"00"
    level_2 = ID[:-4]+"0000"
    level_1 = ID[:-6]+"000000"
    return level_1, level_2, level_3
def get_lv_3(ID):
    return get_parent_from_lv4(ID)[2]
def get_lv_2(ID):
    return get_parent_from_lv4(ID)[1]
def get_lv_1(ID):
    return get_parent_from_lv4(ID)[0]
# ID_list = [itr.IID for itr in tree if itr.level==4]
# test_df = pd.DataFrame({"ID":ID_list})
# test_df["lv_1"] = test_df["ID"].apply(get_lv_1)
# test_df["lv_2"] = test_df["ID"].apply(get_lv_2)
# test_df["lv_3"] = test_df["ID"].apply(get_lv_3)
# test_df

time = get_time_now()
print(time)
with open(PATH_RESULT_OLD_LOGIC, "w") as f:
    f.write("domain_id\tdomain\tIID\tlevel_1\tlevel_2\tlevel_3\ttime\tdate\ttree_version\tlang\n")
for itr in tree:
    if itr.level==4: 
        if len(itr.domains)<1000000:
            ID = itr.IID
            level_1, level_2, level_3 = get_parent_from_lv4(ID)
    #         if len(itr.domains)>1000000:
    #             print("Interest {} - level {} has been skipped".format(itr.file_name, itr.level))
    #         else:
            for i in range(len(itr.domains)):
                domain_id = str(itr.domains.index[i])
                domain = itr.domains.values[i]
                lang = itr.lang.values[i]
                with open(PATH_RESULT_OLD_LOGIC, "a") as f:
                    f.write("\t".join([domain_id, domain, ID, 
                                       level_1, level_2, level_3, time, MONTH, TREE_VERSION, lang])+"\n")
print("Done writing result to file", PATH_RESULT_OLD_LOGIC)

1582598448
Done writing result to file /home/polynote/notebooks/anhdt157/classify-use-ml/end-to-end-final-predict/old_logic/result/result_202001.csv


In [197]:
# output_file = "interest_keywords_v7.csv"
# with open(output_file, 'a') as f:
#     f.write("IID\tinterest_name\tkeywords\n")
#     for itr in tree:
#         interest_name = re.sub(" & ", "_", os.path.join(itr.path, itr.file_name).lower())
#         keywords = ",".join(itr.keywords)
#         f.write("\t".join([itr.IID, interest_name, keywords]))
#         f.write("\n")

In [198]:
df_result = pd.read_csv(PATH_RESULT_OLD_LOGIC, "\t")

In [199]:
# df_result

In [200]:
# 1007900
len(df_result)

988540

In [201]:
df_result = df_result.drop_duplicates(subset=['domain', 'IID'])

In [202]:
# 907868
len(df_result)

988540

In [203]:
df_result.head(5)

,domain_id,domain,IID,level_1,level_2,level_3,time,date,tree_version,lang
0,26,vietdesi.com,A010199,A000000,A010000,A010100,1582598448,202001,8,vi
1,81,inanmocduong.com,A010199,A000000,A010000,A010100,1582598448,202001,8,vi
2,254,hoangkimad.com,A010199,A000000,A010000,A010100,1582598448,202001,8,vi
3,288,inangiahuy.com,A010199,A000000,A010000,A010100,1582598448,202001,8,vi
4,453,inantranhung.com,A010199,A000000,A010000,A010100,1582598448,202001,8,vi


In [204]:
# # for row_test in df_result_test.itertuples():
# print("Writing data to csv file", result_path_distinct)
# overall_start_time = time.time()
# start_time = time.time()
# end_time = time.time()
# count_batch = 0
# batch_size = 10000
# with open(result_path_distinct, 'w', encoding='utf-8') as result_file:
#     f_writer = csv.writer(result_file, delimiter='\t')
#     f_writer.writerow(['domain_id', 'domain', 'IID', 'level_1', 'title_des_raw', 'title_desc_preprocess', 'lang', 'date'])
#     for row in df_result_test.itertuples():
#         count_batch += 1
#         if count_batch % batch_size == 0:
#             print("Loading {}/{}, time to process: {}s, total: {}s"
#                   .format(count_batch,
#                           total_count,
#                           round(time.time() - start_time, 2),
#                           round(time.time() - overall_start_time, 2)))
#             start_time = time.time()
#         f_writer.writerow([row.domain, row.domain_redirect, row.domain_clean, row.redirected,
#                            row.title_des_raw, row.title_desc_preprocess, row.lang, row.date])
# print("Done writing csv data")

# END

In [205]:
# output_test_cant_parsing

In [206]:
#domain_info
len(df_unique)

407720

## Write ra file những domain spam

In [207]:
# list_spam_IID = ['A019999', 'A029999', 'A059999', 'A069999', 'A079999', 'A099999', 'A109999', 
#                  'A119999', 'A999999', 'B019999', 'B999900', 'B999999', 'C999999', 'D010599',
#                  'D029999', 'D049999', 'D059999', 'D069999', 'E029999', 'E999999', 'F999999', 
#                  'G019999', 'G029999', 'G039999', 'G999999', 'H019999', 'H029999', 'H999999']

# list_spam_IID = ['A019999', 'A029999', 'A059999', 'A069999', 'A079999', 'A099999', 'A109999', 
#                  'A119999', 'A999999', 'B019999', 'B999999', 'C999999', 'D010599', 'D059999', 
#                  'D060499', 'D069999', 'E029999', 'E059999', 'E999999', 'F999999', 'F049999', 
#                  'F059999', 'G019999', 'G029999', 'G039999', 'G999999', 'H019999', 'H029999',
#                  'H999999']

list_spam_IID = ['A019999', 'A029999', 'A059999', 'A069999', 'A079999', 'A099999', 'A109999', 
                 'A119999', 'A999999', 'B019999', 'B999999', 'C999999', 'D010599', 'D059999',
                 'D060499', 'D069999', 'D999999', 'E029999', 'E059999', 'E999999', 'F999999',
                 'F049999', 'F059999', 'G019999', 'G029999', 'G039999', 'G999999', 'H019999', 
                 'H029999', 'H999999']

In [208]:
# output_test_spam

In [209]:
# output_test_not_spam

In [210]:
len(df_result)

988540

In [211]:
df_result_all = df_result

In [212]:
for i, value in enumerate(list_spam_IID):
#     print("=======")
#     print(str(value))
    df_result = df_result[~df_result['IID'].astype(str).str.match(value)]
#     print(len(df_result_1))

In [213]:
len(df_result)

96519

In [214]:
len(df_result_all)

988540

In [215]:
# output_test_not_spam
# output_test_spam

In [216]:
# df_result --> not spam

In [217]:
# df_result_all

In [218]:
df_result_all.head(5)

,domain_id,domain,IID,level_1,level_2,level_3,time,date,tree_version,lang
0,26,vietdesi.com,A010199,A000000,A010000,A010100,1582598448,202001,8,vi
1,81,inanmocduong.com,A010199,A000000,A010000,A010100,1582598448,202001,8,vi
2,254,hoangkimad.com,A010199,A000000,A010000,A010100,1582598448,202001,8,vi
3,288,inangiahuy.com,A010199,A000000,A010000,A010100,1582598448,202001,8,vi
4,453,inantranhung.com,A010199,A000000,A010000,A010100,1582598448,202001,8,vi


In [219]:
# df_spam

In [220]:
# df_result_spam = df_result_all[(~df_result_all.domain.isin(df_result.domain)) & (~df_result_all.IID.isin(df_result.IID))]

In [221]:
df_result_all.iloc[[8367]].domain_id

8367    80506
Name: domain_id, dtype: int64

In [222]:
df_spam = df_result_all[~df_result_all.isin(df_result)].dropna()

In [223]:
len(df_spam)

892021

In [224]:
df_spam.iloc[[8367]].domain_id

30565    211685.0
Name: domain_id, dtype: float64

In [225]:
len(df_result)

96519

In [226]:
# 376969 + 59853

### Merge với domain_info để thu được title_desc

In [227]:
#domain_info
len(df_unique)

407720

In [228]:
df_unique.head(5)

,domain_clean,title_desc_preprocess,lang
0,ketoanthuecanthobantin.blogspot.com,bản tin thuế công_ty_tnhh tây nam dịch_vụ báo_...,vi
1,raovattiengiang247.com,rao vat tien_giang mua ban tien_giang rao vat ...,vi
2,onthi.net.vn,cung_cấp kiến_thức lịch_sử việt nam và thế_giớ...,vi
3,xiudunvietnam.net.vn,máy đếm tiền xiudun việt nam giá rẻ trung_tâm ...,vi
4,phukienototanphu.com,đồ_chơi xe_hơi tân phú_ngọc sang chuyên đồ_chơ...,vi


In [229]:
df_info_unique = df_unique.rename(columns={"domain_clean": "domain"})

In [230]:
# df_info_unique

In [231]:
df_info_unique = df_info_unique.drop('lang', axis=1)

In [232]:
type(df_info_unique)

pandas.core.frame.DataFrame

In [233]:
len(df_info_unique)

407720

In [234]:
df_spam = pd.merge(df_spam, df_info_unique, on = 'domain', how = 'inner')
# df_spam = pd.merge(df_spam, df_info_unique, on = 'domain', how = 'inner', indicator=True)

In [235]:
len(df_spam)

892021

In [236]:
df_spam['domain_id'] = df_spam['domain_id'].astype('Int64')
df_spam['tree_version'] = df_spam['tree_version'].astype('Int64')
df_spam['time'] = df_spam['time'].astype('Int64')
df_spam['date'] = df_spam['date'].astype('Int64')

In [237]:
df_spam

,domain_id,domain,IID,level_1,level_2,level_3,time,date,tree_version,lang,title_desc_preprocess
0,516,www.vinapas.com,A029999,A000000,A020000,A029900,1582598448,202001,8,vi,trang_chủ_công_ty thhh thương_mại quốc_tế vina...
1,516,www.vinapas.com,E999999,E000000,E990000,E999900,1582598448,202001,8,vi,trang_chủ_công_ty thhh thương_mại quốc_tế vina...
2,516,www.vinapas.com,F999999,F000000,F990000,F999900,1582598448,202001,8,vi,trang_chủ_công_ty thhh thương_mại quốc_tế vina...
3,8154,baobithiennienky.com,A029999,A000000,A020000,A029900,1582598448,202001,8,vi,công_ty_tnhh thiên_niên_kỷ tuấn anh chuyên sản...
4,8154,baobithiennienky.com,E999999,E000000,E990000,E999900,1582598448,202001,8,vi,công_ty_tnhh thiên_niên_kỷ tuấn anh chuyên sản...
...,...,...,...,...,...,...,...,...,...,...,...
892016,407091,alternativebrewing.com.au,F999999,F000000,F990000,F999900,1582598448,202001,8,en,pour coffee drip coffee alternative brewing al...
892017,407342,nbtrade.com.au,F999999,F000000,F990000,F999900,1582598448,202001,8,en,murwillumbah tweed head business directory fin...
892018,407607,dutchbros.com,F999999,F000000,F990000,F999900,1582598448,202001,8,en,dutch bros coffee shop get dutch bros experien...
892019,3130,khogahoanganh.wordpress.com,G999999,G000000,G990000,G999900,1582598448,202001,8,vi,quà đặc_sản chuyên các loại đặc_sản miền zalo ...


In [238]:
len(df_result)

96519

In [239]:
df_result = pd.merge(df_result, df_info_unique, on = 'domain', how = 'inner')

In [240]:
# df_result.iloc[[257]].tree_version

In [241]:
len(df_result)

96519

### Write to file

In [242]:
# df_spam
# df_result

In [243]:
# 	domain_id	domain	IID	level_1	level_2	level_3	time	date	tree_version	lang

In [244]:
import time
def write_result(path_result, df_write):
    # for row_test in df_result_test.itertuples():
    print("Writing result data to csv file:", path_result)
    overall_start_time = time.time()
    start_time = time.time()
    end_time = time.time()
    count_batch = 0
    batch_size = 10000
    with open(path_result, 'w', encoding='utf-8') as result_file:
        f_writer = csv.writer(result_file, delimiter='|')
        f_writer.writerow(['domain_id', 'domain', 'IID', 
                           'level_1', 'level_2', 'level_3', 
                           'time', 'date','tree_version', 
                           'lang', 'title_desc_preprocess'])
        for row in df_write.itertuples():
            count_batch += 1
            if count_batch % batch_size == 0:
#                 print("Loading {}/{}, time to process: {}s, total: {}s"
#                       .format(count_batch,
#                               total_count,
#                               round(time.time() - start_time, 2),
#                               round(time.time() - overall_start_time, 2)))
                start_time = time.time()
            f_writer.writerow([row.domain_id, row.domain, row.IID, 
                               row.level_1, row.level_2, row.level_3, 
                               row.time, row.date, row.tree_version, 
                               row.lang, row.title_desc_preprocess])
    print("Done writing csv result data")

In [245]:
# Spam file
write_result(output_test_spam, df_spam)

Writing result data to csv file: /home/polynote/notebooks/anhdt157/classify-use-ml/end-to-end-final-predict/train_test_data/test/data-test-spam-202001.csv
Done writing csv result data


In [246]:
# Not spam file
write_result(output_test_not_spam, df_result)

Writing result data to csv file: /home/polynote/notebooks/anhdt157/classify-use-ml/end-to-end-final-predict/train_test_data/test/data-test-not-spam-202001.csv
Done writing csv result data


### READ TEST FILE

In [247]:
df_test_spam_ = pd.read_csv(output_test_spam, delimiter='|')

In [248]:
len(df_test_spam_)

892021

In [249]:
df_test_not_spam_ = pd.read_csv(output_test_not_spam, delimiter='|')

In [250]:
len(df_test_not_spam_)

96519

In [251]:
# type(df_result)
# output_test_not_spam
# output_test_spam

In [252]:
# df_result_test = pd.core.frame.DataFrame()

In [253]:
# type(df_result_test)

In [254]:
# df_result_test

In [255]:
# len(df_result)

In [256]:
# df_result

## Write ra file những domain không parse được

In [257]:
# output_test_cant_parsing

In [258]:
#domain_info
len(df_unique)

407720

In [259]:
#domain_parsed
len(df_result_all)

988540

In [260]:
len(df_unique)

407720

In [261]:
# df_unique

In [262]:
# df_result_all.head(20)

In [263]:
#select only
df_result_distinct = df_result_all
df_result_distinct = df_result_distinct.drop(['domain_id', 'level_1', 'level_2', 
                                         'level_3', 'time', 'date', 'tree_version', 'lang'], axis=1).drop_duplicates(subset=['domain'])
# df_result_spam = df_result_all[(~df_result_all.domain.isin(df_result.domain)) & (~df_result_all.IID.isin(df_result.IID))]

In [264]:
len(df_result_distinct)

407720

In [265]:
len(df_result_all)

988540

In [266]:
df_cant_parsing = df_unique[~df_unique.domain_clean.isin(df_result_distinct.domain)]

In [267]:
df_cant_parsing

,domain_clean,title_desc_preprocess,lang


In [268]:
# df_s = pd.merge(df_spam, df_info_unique, on = 'domain', how = 'inner')

In [269]:
# Write to file - output_test_cant_parsing

In [270]:
len(df_cant_parsing)

0

In [271]:
def write_to_cant_parsing_file(df_cant_parsing, output_file_name):
    print("Writing data to csv file", output_file_name)
    with open(output_file_name, "w") as f:
        f_writer = csv.writer(f, delimiter='|')
        f_writer.writerow(
            ['domain_clean', 'title_desc_preprocess', 'lang'])
        for row in df_cant_parsing.itertuples():
            f_writer.writerow([row.domain_clean, row.title_desc_preprocess, row.lang])
    print("Done writing csv data")

In [272]:
write_to_cant_parsing_file(df_cant_parsing, output_test_cant_parsing)

Writing data to csv file /home/polynote/notebooks/anhdt157/classify-use-ml/end-to-end-final-predict/train_test_data/test/dinfo-data-test-cant-parsing-202001.csv
Done writing csv data


# END